<a href="https://colab.research.google.com/github/El-Eng/freecodecamp/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# import csv data into dataframes
df_train = pd.read_csv(
    train_file_path,
    #encoding = "ISO-8859-1",
    sep="\t",
    header=None,
    names=['class', 'text'],
    usecols=['class', 'text'],
    #dtype={'isbn': 'str', 'title': 'str', 'author': 'str'}
    )

df_test = pd.read_csv(
    test_file_path,
    #encoding = "ISO-8859-1",
    sep="\t",
    header=None,
    names=['class', 'text'],
    usecols=['class', 'text'],
    #dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'}
    )

In [ ]:
df_train['class'] = df_train['class'].map(dict(ham=0, spam=1))
df_test['class'] = df_test['class'].map(dict(ham=0, spam=1))

In [ ]:
train_labels = df_train.pop('class')
test_labels = df_test.pop('class')

In [ ]:
print(df_train['text'].str.len().max())
print(df_test['text'].str.len().max())

In [ ]:
from tensorflow.keras.layers import TextVectorization
# Model constants.
embedding_dim = 128
sequence_length = 910

vectorize_layer = TextVectorization(
    standardize="lower",
    output_mode="int",
    output_sequence_length=sequence_length,
)

In [ ]:
vectorize_layer.adapt(df_train.text.to_list())

In [ ]:
from tensorflow.keras import layers

# ref = https://keras.io/examples/nlp/text_classification_from_scratch/

text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
x = vectorize_layer(text_input)
x = layers.Embedding(20000, embedding_dim )(x)

x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(text_input, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
epochs = 3

# Fit the model using the train and test datasets.
model.fit(df_train['text'].to_list(), train_labels.to_list(), epochs=epochs)

In [ ]:
model.evaluate(df_test['text'].to_list(), test_labels.to_list(), verbose=2)

In [ ]:
res = model.predict(["how are you doing today"])


In [ ]:
res[0][0]

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  res = model.predict([pred_text])
  val = res[0][0]
  if val>0.5:
    pred = "spam"
  else:
    pred = "ham"
  prediction = [val, pred]
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
